In [188]:
#libraries
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine, euclidean, hamming
from sklearn.preprocessing import normalize

In [189]:
#read the data & have randome sample for reproducability
recipes = pd.read_csv('../data/full_clean_data.csv')
recipes.shape

(19224, 14)

In [190]:
##rename the dataframe index
recipes.index.names = ['recipes_id']
##convert index to column
recipes['recipes_id'] = recipes.index
recipes.tail(1)

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed,recipes_id
recipes_id,,,,,,,,,,,,,,,
19223,Easy Baked Zucchini Chips,https://www.allrecipes.com/recipe/242066/easy-...,appetizers-and-snacks,3.68,28,"['1 zucchini, thinly sliced ', ' 1 tablespoon...",30,2,2 servings,69.1,2.0,6.9,0.7,['flavored'],19223


In [191]:
##normalize the numerical column (total+nutritional values) in new dataframe << because i dont want to normalize all other features
newrecipes = recipes[['total', 'calories', 'carbohydrates_g', 'fat_g', 'protein_g']]
recipes_norm = pd.DataFrame(normalize(newrecipes, axis=0))
recipes_norm.columns = newrecipes.columns
recipes_norm.index = newrecipes.index
recipes_norm.tail(2)

,total,calories,carbohydrates_g,fat_g,protein_g
recipes_id,,,,,
19222,0.000419,0.001771,0.001872,0.001735,0.001107
19223,0.001256,0.001640,0.000412,0.003421,0.000298


In [192]:
#####creare a function that recommends top 3 similar recipe. Guidance reference https://www.kaggle.com/code/yyzz1010/content-based-filtering-recipe-recommender?scriptVersionId=21219821&cellId=23
#####tried 3 different functions (cosine, euclidean & hamming) on the same recipe id & found out that (euclidean) recommends the closest recipes.
def recipe_recommender(recipes_id):
    
     allRecipes = pd.DataFrame(recipes_norm.index)
     allRecipes = allRecipes[allRecipes.recipes_id != recipes_id]
     allRecipes["distance"] = allRecipes["recipes_id"].apply(lambda x: euclidean(recipes_norm.loc[recipes_id], recipes_norm.loc[x]))
     # sort by distance then recipe id, the smaller value will be picked.
     Top3NRecommendation = allRecipes.sort_values(["distance"]).head(3).sort_values(by=['distance', 'recipes_id'])
     #print the distance value
     print(Top3NRecommendation)
     #create a list of the recommended recipe output
     recipes_id = [recipes_id]
     recipe_list = [] #'name', 'url', 'total cooking time', 'cal', 'carbs', 'fat','protein'
     for recipeid in Top3NRecommendation.recipes_id:
        recipes_id.append(recipeid)   # list of recipe id of selected recipe and recommended recipe
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'name']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'url']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'total']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'calories']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'carbohydrates_g']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'fat_g']))
        recipe_list.append("{}  {}".format(recipeid, recipes.at[recipeid, 'protein_g']))
     return recipe_list

In [193]:
##recipe id we want to get similar recommendation to
recipes[recipes['recipes_id'] == 4594]

,name,url,category,rating,rating_count,ingredients,total,servings,yield,calories,carbohydrates_g,fat_g,protein_g,ingredients_parsed,recipes_id
recipes_id,,,,,,,,,,,,,,,
4594,Potato Waffles,https://www.allrecipes.com/recipe/220890/potat...,breakfast-and-brunch,4.57,112,"['2 tablespoons butter ', ' 1 onion, chopped ...",25,4,4 servings,216.7,27.9,9.0,6.5,"['butter', 'mashed potato', 'flour', 'egg']",4594


In [194]:
#recommend the top 3 most similar recipes based on (total, calories, protein, carbs, fat)
recipe_recommender(4594)

       recipes_id  distance
17848       17848  0.000369
2645         2645  0.000536
648           648  0.000576


['17848  Easy Chilaquiles Verdes',
 '17848  https://www.allrecipes.com/recipe/271116/easy-chilaquiles-verdes/',
 '17848  20',
 '17848  216.3',
 '17848  26.8',
 '17848  9.4',
 '17848  6.6',
 '2645  Fereni Starch Pudding',
 '2645  https://www.allrecipes.com/recipe/16079/fereni-starch-pudding/',
 '2645  20',
 '2645  222.9',
 '2645  28.3',
 '2645  9.9',
 '2645  6.2',
 '648  My Crispy Mashed Potato Pancake',
 '648  https://www.allrecipes.com/recipe/222608/my-crispy-mashed-potato-pancake/',
 '648  15',
 '648  214.2',
 '648  26.9',
 '648  8.7',
 '648  7.2']